In [ ]:
from flypipe import node
from flypipe.datasource.spark import Spark

@node(
    type="pyspark",
    dependencies=[Spark("bronze")],
    group="data_pipeline"
)
def silver(bronze):
    return bronze


@node(
    type="pyspark",
    dependencies=[silver],
    group="data_pipeline"
)
def gold(silver):
    return silver

@node(
    type="pandas_on_spark",
    group="feature_pipeline",
    dependencies=[gold]
)
def feature1(gold):
    return gold

@node(
    type="pandas_on_spark",
    group="feature_pipeline",
    dependencies=[gold]
)
def feature2(gold):
    return gold


@node(
    type="pandas_on_spark",
    group="feature_pipeline",
    dependencies=[gold]
)
def feature3(gold):
    return gold


@node(
    type="pandas_on_spark",
    dependencies=[
        feature1, feature2, feature3
    ]
)
def model_features(feature1, feature2, feature3):
    return feature1


@node(
    type="pandas_on_spark",
    group="model_training",      
    dependencies=[model_features]
)
def split(model_features):
    return model_features


@node(
    type="pandas_on_spark",
    group="model_training",
    dependencies=[split]
)
def train_scale(split):
    return split

@node(
    type="pandas",
    group="model_training",
    dependencies=[train_scale]
)
def model_train(train_scale):
    return train_scale


@node(
    type="pandas",
    group="model_training",
    dependencies=[model_train]
)
def evaluate(model_train):
    return model_train

@node(
    type="pandas",
    group="prediction",
    dependencies=[model_features]
)
def scale(model_features):
    return model_features


@node(
    type="pandas",
    group="prediction",
    dependencies=[scale]
)
def predict(scale):
    return scale


@node(
    type="pandas",
    dependencies=[predict, evaluate]
)
def my_graph(predict, evaluate):
    return predict

displayHTML(my_graph.html())